# Characters — Notes Companion

**6.7970/8.750 Symmetry and its Application to Machine Learning**

This notebook accompanies the [Characters notes](https://symm4ml.mit.edu/notes/characters). Run these examples to build character tables and decompose reducible representations.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atomicarchitects/symm4ml-colabs/blob/main/characters_notes_companion.ipynb)

## Setup

In [ ]:
%%capture
!pip install https://symm4ml.mit.edu/_static/symm4ml_s26/symm4ml/symm4ml_latest.zip

In [ ]:
import numpy as np
from symm4ml import groups, linalg, rep

## Reference: $P(3)$ data

In [ ]:
# Build P(3) table, find irreps, and get conjugacy classes
p3_matrices = groups.permutation_matrices(3)
table = groups.make_multiplication_table(p3_matrices)
irreps = rep.infer_irreps(table)
conj_classes = groups.conjugacy_classes(table)

print(f"P(3): h = {len(table)}, {len(irreps)} irreps, {len(conj_classes)} conjugacy classes")
print(f"Irrep dimensions: {[ir.shape[1] for ir in irreps]}")
print(f"Conjugacy classes: {conj_classes}")

---
## Example: Building the Character Table

The **character** of a representation is the trace: $\chi(g) = \text{tr}(D(g))$.
Since the trace is invariant under similarity transforms, the character captures the essential information of a representation.

The function `rep.character_table(irreps, conj_classes)` computes the full character table.

In [ ]:
# Build the character table
char_table = rep.character_table(irreps, conj_classes)
print("Character table of P(3):")
print(f"  Rows = irreps, Columns = conjugacy classes\n")
print(np.round(char_table.real, 4))

In [ ]:
# The first column (identity class) gives the dimension of each irrep:
# \chi(E) = tr(I_{\ell_j}) = \ell_j
print("\u03c7(E) for each irrep (= dimension):")
for i, ir in enumerate(irreps):
    print(f"  \u0393_{i+1}: \u03c7(E) = {char_table[i, 0].real:.0f}  (dim = {ir.shape[1]})")

In [ ]:
# Verify the Wonderful Orthogonality Theorem holds
print(f"WOT satisfied? {rep.check_orthogonality_theorem(irreps)}")

---
## Example: Decomposing the 3D Permutation Representation

The 3D permutation matrices of $P(3)$ form a **reducible** representation.
The notes derive by hand that it decomposes as $\Gamma_1 \oplus \Gamma_2$.
Let's verify this computationally.

In [ ]:
# Build the 3D permutation representation
perm_rep = groups.permutation_matrices(3)  # shape [6, 3, 3]

# Is it irreducible?
print(f"Permutation rep dimension: {perm_rep.shape[1]}")
print(f"Is it an irrep? {rep.is_an_irrep(table, perm_rep)}")

In [ ]:
# Decompose into irreps
sub_irreps = rep.decompose_rep_into_irreps(perm_rep)

print(f"Decomposes into {len(sub_irreps)} irreps:")
for i, sub in enumerate(sub_irreps):
    print(f"  Sub-irrep {i+1}: dimension {sub.shape[1]}")

In [ ]:
# Identify which known irreps they correspond to
print("Identifying sub-irreps:")
for i, sub in enumerate(sub_irreps):
    for j, known in enumerate(irreps):
        if rep.are_isomorphic(sub, known):
            print(f"  Sub-irrep {i+1} \u2245 \u0393_{j+1} (dim {known.shape[1]})")
            break

print("\nThis matches the hand calculation: perm rep = \u0393_1 \u2295 \u0393_2")